## Online Evaluators

때때로 프로젝트에 기록된 생성물을 평가하고자 할 때가 있습니다.

In [ ]:
# 설치
# !pip install -qU langsmith langchain-teddynote

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Auto-Evaluation-Test")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Auto-Evaluation-Test


## 온라인 Evaluation을 위한 Chain 설정

In [3]:
from myrag import PDFRAG
from langchain_openai import ChatOpenAI

# PDFRAG 객체 생성
rag = PDFRAG(
    "data/SPRI_AI_Brief_2023년12월호_F.pdf",
    ChatOpenAI(model="gpt-4o-mini", temperature=0),
)

# 검색기(retriever) 생성
retriever = rag.create_retriever()

# 체인(chain) 생성
rag_chain = rag.create_chain(retriever)

In [4]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# RunnableParallel 객체 생성합니다.
evaluation_runnable = RunnableParallel(
    {
        "context": retriever,
        "answer": rag_chain,
        "question": RunnablePassthrough(),
    }
)

테스트 체인을 실행하여 Runs 에 결과가 반영되는지 확인합니다.

In [5]:
# 체인 실행
_ = evaluation_runnable.invoke("삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?")

## Online LLM-as-judge 생성

## 참고 URL : https://docs.langchain.com/langsmith/online-evaluations#configure-llm-as-judge-evaluators

> Online LLM-as-judge 생성

![](./assets/1.png)


![](./assets/2.png)


![](./assets/3.png)


![](./assets/4.png)


![](./assets/5.png)


![](./assets/6.png)


![](./assets/7.png)

In [6]:
from langchain_core.runnables import RunnableConfig

# 태그를 설정합니다.
hallucination_config = RunnableConfig(tags=["hallucination_eval"])
context_recall_config = RunnableConfig(tags=["context_recall_eval"])
all_eval_config = RunnableConfig(tags=["hallucination_eval", "context_recall_eval"])

In [7]:
# 체인 실행
_ = evaluation_runnable.invoke("삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?")

In [8]:
# Hallucination 평가 요청
_ = evaluation_runnable.invoke(
    "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?", config=hallucination_config
)

In [25]:
# Context Recall 평가 요청
_ = evaluation_runnable.invoke(
    "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?",
    config=context_recall_config,
)

In [31]:
# 모든 평가 요청
_ = evaluation_runnable.invoke(
    "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?", config=all_eval_config
)



![](./assets/8.png)


![](./assets/9.png)

